In [133]:
#importing libraries

In [134]:
import numpy as np
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPool2D

from sklearn.neighbors import NearestNeighbors
import os
from numpy.linalg import norm

In [135]:
#Extract filenames from folder

In [136]:
filenames = []
for file in os.listdir('images'):
    filenames.append(os.path.join('images',file))

In [137]:
len(filenames)

44441

In [138]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
model.trainable = False

model = tf.keras.models.Sequential([model,
                                   GlobalMaxPool2D()
                                   ])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling2d_2               │ (None, 2048)                │               0 │
│ (GlobalMaxPooling2D)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [139]:
img = image.load_img('16871.jpg', target_size=(224,224))
img_array = image.img_to_array(img)
img_expand_dim = np.expand_dims(img_array, axis=0)
img_preprocess = preprocess_input(img_expand_dim)
result = model.predict(img_preprocess).flatten()
norm_result = result/norm(result)
norm_result

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([0.        , 0.00295298, 0.        , ..., 0.00752506, 0.02113524,
       0.0040174 ], dtype=float32)

In [140]:
def extract_features_from_images(image_path, model):
    img = image.load_img(image_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_expand_dim = np.expand_dims(img_array, axis=0)
    img_preprocess = preprocess_input(img_expand_dim)
    result = model.predict(img_preprocess).flatten()
    norm_result = result/norm(result)
    return norm_result

In [141]:
extract_features_from_images(filenames[0], model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step


array([0.        , 0.01761618, 0.00171608, ..., 0.01247239, 0.02726405,
       0.06899218], dtype=float32)

In [142]:
image_features = []
for file in filenames:
    image_features.append(extract_features_from_images(file, model))
image_features

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


[array([0.        , 0.01761618, 0.00171608, ..., 0.01247239, 0.02726405,
        0.06899218], dtype=float32),
 array([0.        , 0.03648944, 0.        , ..., 0.00997913, 0.02375535,
        0.04649905], dtype=float32),
 array([0.        , 0.03507714, 0.00732278, ..., 0.00283509, 0.        ,
        0.05435881], dtype=float32),
 array([0.00232165, 0.05030549, 0.00747743, ..., 0.00346684, 0.03391018,
        0.04565744], dtype=float32),
 array([0.00306834, 0.0624045 , 0.        , ..., 0.00170625, 0.02032885,
        0.0583326 ], dtype=float32)]

In [143]:
Image_features = pkl.dump(image_features, open('Images_features.pkl','wb'))

In [144]:
filenames = pkl.dump(filenames, open('filenames.pkl','wb'))

In [145]:
#Loading Pickle Files

In [146]:
Image_features = pkl.load(open('Images_features.pkl','rb'))

In [147]:
filenames = pkl.load(open('filenames.pkl','rb'))

In [148]:
np.array(Image_features).shape

(5, 2048)

In [149]:
neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')

In [150]:
neighbors.fit(Image_features)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [151]:
input_image = extract_features_from_images('16871.jpg',model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step


In [152]:
distance,indices = neighbors.kneighbors([input_image])

ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 5, n_samples = 1

In [ ]:
indices[0]

In [ ]:
from IPython.display import Image

In [ ]:
Image('16871.jpg')

In [ ]:
Image(filenames[indices[0][1]])

In [ ]:
Image(filenames[indices[0][2]])

In [ ]:
Image(filenames[indices[0][3]])

In [ ]:
Image(filenames[indices[0][4]])

In [ ]:
Image(filenames[indices[0][5]])